In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
import gymnasium as gym

from astropy.io import fits
from datetime import datetime
import json
# import fitsio
import pandas as pd
import time
import pickle
import re


%reload_ext autoreload
%autoreload 2

In [2]:
import survey_ops
from survey_ops.utils import units, geometry, interpolate
from survey_ops.utils import ephemerides
from survey_ops.src.offline_dataset import OfflineDECamDataset
from survey_ops.src.agents import Agent
from survey_ops.src.algorithms import DDQN, BehaviorCloning
from survey_ops.utils.pytorch_utils import seed_everything


In [3]:
SEED = 10
seed_everything(SEED)
torch.set_default_dtype(torch.float32)
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"   
)

In [4]:
def relabel_mislabelled_objects(df):
    """Renames object columns with 'object_name (outlier)' if they are outside of a certain cutoff from the median RA/Dec.

    Args
    ----
    df (pd.DataFrame): The dataframe with object names and RA/Dec positions.

    Returns
    -------
    df_relabelled (pd.DataFrame): The dataframe with relabelled objects.
    """

    object_radec_df = df[['object', 'ra', 'dec']]
    object_radec_groups = object_radec_df.groupby('object')
    df_relabelled = df.copy(deep=True)

    outlier_indices = []
    for _, g in object_radec_groups:
        cutoff_deg = 3
        median_ra = g.ra.median()
        delta_ra = g.ra - median_ra
        delta_ra_shifted = np.remainder(delta_ra + 180, 360) - 180
        mask_outlier_ra = np.abs(delta_ra_shifted) > cutoff_deg

        median_dec = g.dec.median()
        delta_dec = g.dec - median_dec
        delta_dec_shifted = np.remainder(delta_dec + 180, 360) - 180
        mask_outlier_dec = np.abs(delta_dec_shifted) > cutoff_deg

        mask_outlier = mask_outlier_ra | mask_outlier_dec

        if np.count_nonzero(mask_outlier) > 0:
            indices = g.index[mask_outlier].values
            outlier_indices.extend(indices)
    df_relabelled.loc[outlier_indices, 'object'] = [f'{obj_name} (outlier)' for obj_name in df.loc[outlier_indices, 'object'].values]
    return df_relabelled

In [5]:
def save_bin_and_field_mappings(
    nside,
    bin_space='radec',
    specific_years=None, 
    specific_months=None, 
    specific_days=None, 
    specific_filters=None,
    data_json_fn=None,
    data_fits_fn=None,
    outdir='../data/',
    bin2radec_fn = 'bin2radec.json',
    field2radec_fn = 'field2radec.json',
    bin2fields_in_bin_fn = 'bin2fields_in_bin.json',
    field2name_fn = 'field2name.json',
    field2nvisits_fn = 'field2nvisits.json'
):
    if data_json_fn is None:
        data_json_fn = '../data/data_w_dith.json'
    if data_fits_fn is None:
        data_fits_fn = '../data/decam-exposures-20251211_w_dith.fits'
    
    try:
        # --- Load json df ---- #
        df = pd.read_json(data_json_fn)
        print('Loaded data from json')
    except:
        # --- Load fits ---- #
        print(data_json_fn, 'DNE. Loading and processing data from fits.')
        d = fitsio.read(data_fits_fn)
        sel = (d['propid'] == '2012B-0001') & (d['exptime'] > 40) & (d['exptime'] < 100) & (~np.isnan(d['teff']))
        selected_d = d[sel]
        column_names = selected_d.dtype.names
        df = pd.DataFrame(selected_d, columns=column_names)
        
        # mask = np.array([False if 'None' in dt else True for dt in subs_data['datetime']])
        # subs_data = {key: subs_data[key][mask] for key in subs_data.keys()}
        
        df.to_json(data_json_fn, indent=2)
        df = pd.read_json(data_json_fn)

    df['night'] = (df['datetime'] - pd.Timedelta(hours=12)).dt.normalize()

    # Get observations for specific years, days, filters, etc.
    if specific_years is not None:
        df = df[df['night'].dt.year.isin(specific_years)]
    if specific_months is not None:
        df = df[df['night'].dt.month.isin(specific_months)]
    if specific_days is not None:
        df = df[df['night'].dt.day.isin(specific_days)]
    if specific_filters is not None:
        df = df[df['filter'].isin(specific_filters)]
        
    df = df[df['night'].dt.year != 1970]
    assert len(df) > 0, "No observations found for the specified year/month/day/filter selections."

    df = relabel_mislabelled_objects(df)

    # Add timestamp col
    utc = pd.to_datetime(df['datetime'], utc=True)
    timestamps = (utc.astype('int64') // 10**9).values
    df['timestamp'] = timestamps
    
    # Sort df by timestamp
    df = df.sort_values(by='timestamp')

    # convert degrees to radians
    df['el'] = np.pi/2 - df['zd'].values
    df.loc[:, ['ra', 'dec', 'az', 'el', 'zd']] *= units.deg

    hpGrid = ephemerides.HealpixGrid(nside=nside, is_azel=(bin_space == 'azel'))

    # Add bin column to dataframe
    if hpGrid.is_azel:
        lon = df['az']
        lat = df['el']
        # time x ra, dec --> bin: (az, el)
        # self.bin2fields = lambda time: ephemerides.equatorial_to_topographic(ra=df['az'], dec=df['el'], time=time)
        # raise NotImplementedError
    else:
        lon = df['ra']
        lat = df['dec']
        df['bin'] = hpGrid.ang2idx(lon=lon, lat=lat)

    df['field_id'] = pd.factorize(df['object'])[0]

    bin2radec = {int(i): (lon, lat) for i, (lon, lat) in zip(hpGrid.heal_idx, zip(hpGrid.lon, hpGrid.lat))}
    field2radec = {int(field_id): g.loc[:, ['ra', 'dec']].mean(axis=0).values.tolist() for field_id, g in df.groupby('field_id')}
    bin2fields_in_bin = {int(bin_id): g['field_id'].values.tolist() for bin_id, g in df.groupby('bin')}
    field2name = {field_id: g.loc[:, ['object']].values.tolist()[0][0] for field_id, g in df.groupby('field_id')}
    field_ids, counts = np.unique(df['field_id'], return_counts=True)
    field2nvisits = {int(fid): int(count) for fid, count in zip(field_ids, counts)}
    # return field2nvisits
    # field2radec = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in field2radec.items()}
    # bin2fields_in_bin = {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in bin2fields_in_bin.items()}
    
    with open(outdir + f'nside{nside}_' + bin2radec_fn, "w") as f:
        json.dump(bin2radec, f)

    with open(outdir + field2radec_fn, "w") as f:
        json.dump(field2radec, f)

    with open(outdir + f'nside{nside}_' + bin2fields_in_bin_fn, "w") as f:
        json.dump(bin2fields_in_bin, f)
        
    with open(outdir + field2name_fn, "w") as f:
        json.dump(field2name, f)

    with open(outdir + field2nvisits_fn, "w") as f:
        json.dump(field2nvisits, f)
    return

In [8]:
nside=32
bin_space='radec'
# specific_years=[2014]
# specific_months=[12]
# specific_days=[15, 16, 17, 18, 19]
specific_years=None
specific_months = None
specific_days=None

In [9]:
field2nvisits = save_bin_and_field_mappings(
    nside=nside, 
    bin_space=bin_space, 
    specific_years=specific_years, 
    specific_months=specific_months, 
    specific_days=specific_days
)

Loaded data from json


In [43]:
with open('../data/field2radec.json', 'r') as f:
    data = json.load(f)

In [44]:
len(data)

16266